In [1]:
print("hello")

hello


In [2]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options) 

driver.get('https://www.google.com/')
print(driver.title)

search_box = driver.find_element_by_name("q")
search_box.send_keys('ChromeDriver')
search_box.submit()
print(driver.title)

driver.save_screenshot('search_results_re.png')
driver.quit()

Google


/tmp/ipykernel_275/504579241.py:11: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  search_box = driver.find_element_by_name("q")


ChromeDriver - Google 検索


In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)
q = u"ワールドカップ 2018"
driver.get('https://www.google.com/search?q=%s' % (q))